<a href="https://colab.research.google.com/github/nalivaikaanastasiya-dev/HTTP-Web-Search-Briefing-Bot/blob/main/server_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
from typing import List
from fastapi import FastAPI, Header, HTTPException
from pydantic import BaseModel, Field
import requests

# ================= CONFIG =================
MCP_HTTP_TOKEN = os.getenv("MCP_HTTP_TOKEN", "devtoken123")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")

if not TAVILY_API_KEY:
    raise RuntimeError("Set TAVILY_API_KEY")

app = FastAPI(title="HTTP Web Search Briefing Bot")

# ================= AUTH =================
def check_auth(auth: str | None):
    if auth != f"Bearer {MCP_HTTP_TOKEN}":
        raise HTTPException(status_code=401, detail="Unauthorized")

# ================= MODELS =================
class SearchReq(BaseModel):
    query: str
    k: int = Field(default=5, le=10)

class FetchReq(BaseModel):
    url: str

class SummarizeReq(BaseModel):
    topic: str
    docs: List[dict]

class SaveReq(BaseModel):
    filename: str
    content: str

# ================= TOOLS =================
@app.get("/tools")
def list_tools(authorization: str = Header(None)):
    check_auth(authorization)
    return {
        "tools": {
            "search_web": {"query": "str", "k": "int"},
            "fetch_readable": {"url": "str"},
            "summarize_with_citations": {"topic": "str", "docs": "list"},
            "save_markdown": {"filename": "str", "content": "str"},
        }
    }

@app.post("/tools/search_web")
def search_web(req: SearchReq, authorization: str = Header(None)):
    check_auth(authorization)
    r = requests.post(
        "https://api.tavily.com/search",
        json={
            "api_key": TAVILY_API_KEY,
            "query": req.query,
            "max_results": req.k,
        },
        timeout=15,
    )
    r.raise_for_status()
    results = r.json()["results"]
    return [
        {
            "title": x["title"],
            "url": x["url"],
            "snippet": x.get("content", ""),
            "source": "tavily",
        }
        for x in results
    ]

@app.post("/tools/fetch_readable")
def fetch_readable(req: FetchReq, authorization: str = Header(None)):
    check_auth(authorization)
    r = requests.get(req.url, timeout=15)
    r.raise_for_status()
    return {
        "url": req.url,
        "title": req.url,
        "text": r.text[:8000],
    }

@app.post("/tools/summarize_with_citations")
def summarize(req: SummarizeReq, authorization: str = Header(None)):
    check_auth(authorization)

    prompt = f"""
Topic: {req.topic}

Documents:
{req.docs}

Rules:
- Exactly 5 bullets
- Max 200 chars each
- Inline citations like [1], [2]
"""

    r = requests.post(
        f"{OLLAMA_BASE_URL}/api/chat",
        json={
            "model": "llama3",
            "messages": [{"role": "user", "content": prompt}],
            "stream": False,
        },
        timeout=30,
    )
    r.raise_for_status()
    text = r.json()["message"]["content"]

    bullets = [b.strip("- ") for b in text.split("\n") if b.strip()][:5]

    sources = [
        {"i": i + 1, "title": d["title"], "url": d["url"]}
        for i, d in enumerate(req.docs)
    ]

    return {"bullets": bullets, "sources": sources}

@app.post("/tools/save_markdown")
def save_md(req: SaveReq, authorization: str = Header(None)):
    check_auth(authorization)
    path = os.path.abspath(req.filename)
    with open(path, "w", encoding="utf-8") as f:
        f.write(req.content)
    return {"path": path}

